# YOLOv4 + DeepFashion2 Datasets 로 Object Detection 하기

다음 자료들을 참고하여 공부하고 코드를 작성하였습니다.
* Use YOLOv4 to train the DeepFashion2 Dataset ([문서](https://www.programmersought.com/article/74557705857/))
* Darknet 학습 준비하기 ([블로그](https://eehoeskrap.tistory.com/367))
* YOLOv4 custom 데이터 훈련하기 ([블로그](https://keyog.tistory.com/22?category=879585))
* Colab에서 Detectron2로 의류 인식하기 ([블로그](https://airsbigdata.tistory.com/208))
* ssaru/convert2Yolo ([GitHub](https://github.com/ssaru/convert2Yolo))

YOLO는 Darknet이라는 프레임워크를 사용합니다.  
Darknet은 YOLOv4 뿐 아니라 다양한 버전의 YOLO 모델을 서빙하고 있습니다.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15050, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 15050 (delta 16), reused 34 (delta 14), pack-reused 15002
Receiving objects: 100% (15050/15050), 13.48 MiB | 16.65 MiB/s, done.
Resolving deltas: 100% (10215/10215), done.


해당 Darknet Repository의 [README.md](https://github.com/AlexeyAB/darknet#how-to-evaluate-fps-of-yolov4-on-gpu) 에서는 GPU를 사용한 YOLOv4 를 학습하기 전 Darknet 프레임워크를 초기화(빌드)하는 방법을 설명하고 있습니다.  darknet/Makefile 을 다음과 같이 수정하고 아래 셀의 코드를 실행하여 반영합니다:  
```
# Makefile 의 앞부분
GPU=1
CUDNN=1 
CUDNN_HALF=1 
OPENCV=1
LIBSO=1
```

In [ ]:
%cd darknet/
!make
!mkdir DeepFashion2
%cd ../

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if 

본인의 Google Drive 를 마운트하여 [데이터셋](https://drive.google.com/drive/folders/125F48fsMBz2EF0Cpqk6aaHet5VH399Ok)을 복사합니다.  
[DeepFashion2](https://github.com/switchablenorms/DeepFashion2) 를 본인의 Google Drive 에 받는 자세한 방법은 [링크](https://beausty23.tistory.com/82)를 참고하세요.  
DeepFashion2 의 데이터셋이 매우 방대하므로 이 학습에서는 Validation Set 만을 받아 Train 과 Validation으로 나누어 사용합니다.   


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp /content/gdrive/MyDrive/validation.zip /content/darknet/DeepFashion2/validation.zip

패스워드와 함께 데이터셋의 압축을 해제합니다.

In [ ]:
!unzip -P "패스워드" /content/darknet/DeepFashion2/validation.zip -d /content/darknet/DeepFashion2/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/darknet/DeepFashion2/validation/image/007349.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007310.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007286.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007274.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007218.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007216.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007204.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007133.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007081.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007045.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007044.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/007017.jpg  
  inflating: /content/darknet/DeepFashion2/validation/image/006960.jpg  
  inflating: /c

다음은 클래스를 나누어 각각의 이미지와 그에 대응하는 클래스, 어노테이션을 묶는 validation.json 파일을 생성하는 코드입니다.  
이 작업이 필요한 이유는 현재 DeepFashion2 의 Annotation Format 이 YOLOv4 가 요구하는 Format 과 맞지 않기 때문입니다.  
다음과 같은 순서로 Format 을 맞출 예정입니다.
1. DeepFashion2 의 Annotation Format 을  COCO Format 으로 변환
2. COCO Format 을 YOLO Format 으로 변환  

아래 셀의 코드는 1.의 과정입니다.  
DeepFashion2 의 데이터셋은 총 13개 클래스입니다.

In [ ]:
!mkdir /content/darknet/DeepFashion2/validation/annotations

In [ ]:
from PIL import Image
import numpy as np
import json

dataset = {
    "info": {},
    "licenses": [],
    "images": [],
    "annotations": [],
    "categories": []
}

lst_name = ['short_sleeved_shirt', 'long_sleeved_shirt', 'short_sleeved_outwear', 'long_sleeved_outwear',
            'vest', 'sling', 'shorts', 'trousers', 'skirt', 'short_sleeved_dress',
            'long_sleeved_dress', 'vest_dress', 'sling_dress']

for idx, e  in enumerate(lst_name):
    dataset['categories'].append({
        'id': idx + 1,
        'name': e,
        'supercategory': "clothes",
        'keypoints': ['%i' % (i) for i in range(1, 295)],
        'skeleton': []
    })

num_images = 32153 #191961 
sub_index = 0  # the index of ground truth instance
for num in range(1, num_images + 1):
    json_name = '/content/DeepFashion2/validation/annos/' + str(num).zfill(6) + '.json'
    image_name = '/content/DeepFashion2/validation/image/' + str(num).zfill(6) + '.jpg'

    if (num >= 0):
        imag = Image.open(image_name)
        width, height = imag.size
        with open(json_name, 'r') as f:
            temp = json.loads(f.read())
            pair_id = temp['pair_id']

            dataset['images'].append({
                'coco_url': '',
                'date_captured': '',
                'file_name': str(num).zfill(6) + '.jpg',
                'flickr_url': '',
                'id': num,
                'license': 0,
                'width': width,
                'height': height
            })
            for i in temp:
                if i == 'source' or i == 'pair_id':
                    continue
                else:
                    points = np.zeros(294 * 3)
                    sub_index = sub_index + 1
                    box = temp[i]['bounding_box']
                    w = box[2] - box[0]
                    h = box[3] - box[1]
                    x_1 = box[0]
                    y_1 = box[1]
                    bbox = [x_1, y_1, w, h]
                    cat = temp[i]['category_id']
                    style = temp[i]['style']
                    seg = temp[i]['segmentation']
                    landmarks = temp[i]['landmarks']

                    points_x = landmarks[0::3]
                    points_y = landmarks[1::3]
                    points_v = landmarks[2::3]
                    points_x = np.array(points_x)
                    points_y = np.array(points_y)
                    points_v = np.array(points_v)
                    case = [0, 25, 58, 89, 128, 143, 158, 168, 182, 190, 219, 256, 275, 294]
                    idx_i, idx_j = case[cat - 1], case[cat]

                    for n in range(idx_i, idx_j):
                        points[3 * n] = points_x[n - idx_i]
                        points[3 * n + 1] = points_y[n - idx_i]
                        points[3 * n + 2] = points_v[n - idx_i]

                    num_points = len(np.where(points_v > 0)[0])

                    dataset['annotations'].append({
                        'area': w * h,
                        'bbox': bbox,
                        'category_id': cat,
                        'id': sub_index,
                        'pair_id': pair_id,
                        'image_id': num,
                        'iscrowd': 0,
                        'style': style,
                        'num_keypoints': num_points,
                        'keypoints': points.tolist(),
                        'segmentation': seg,
                    })

json_name = '/content/darknet/DeepFashion2/validation/validation.json'
with open(json_name, 'w') as f:
    json.dump(dataset, f)

100%|██████████| 30000/30000 [01:19<00:00, 378.48it/s]


In [ ]:
%cd /content/darknet/DeepFashion2
!mkdir YOLO/

/content/darknet/DeepFashion2


아래 셀의 코드는 2.의 과정입니다.  
DeepFashion2 폴더 내에 다음 2개의 파일을 추가한 후 실행해야 합니다.
1. [example.py](https://github.com/ssaru/convert2Yolo/blob/master/example.py)
2. [Format.py](https://github.com/ssaru/convert2Yolo/blob/master/Format.py)

In [ ]:
!python ./example.py --datasets COCO --img_path ./validation/image/ --label ./validation/annotations/validation.json --convert_output_path ./YOLO/ --img_type ".jpg" --manifest_path ./validation/ --cls_list_file ./validation/deepfashion2.names


COCO Parsing:  |████████████████████████████████████████| 100.0% (48715/48715)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (30000/30000)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (30000/30000)  Complete



위 작업이 끝나면 다음의 것들이 생성됩니다.
- YOLO/*.txt : 이미지별 바운딩 좌표가 Yolo Format 으로 변환되어 있습니다.
- manifest.txt : 모든 이미지의 경로를 모아 놓은 파일입니다.  

이후 데이터를 shuffle 하는 코드입니다.

In [ ]:
import random 
txt = open('/content/darknet/DeepFashion2/validation/manifest.txt','r') 
f = open('/content/darknet/DeepFashion2/validation/manifest_shuffle.txt','w') 
tmp = [] 
while True : 
  line = txt.readline() 
  if not line: 
    break 
  tmp.append(line) 
random.shuffle(tmp) 
for i in tmp : 
  f.write(i) 
txt.close() 
f.close()

shuffle 된 데이터셋을 Train 과 Validation 으로 나누는 코드입니다.

In [ ]:
count = 0 
length = 37205 #total line 
txt = open('/content/darknet/DeepFashion2/validation/manifest_shuffle.txt','r') 
i = 0 
f = open('/content/darknet/custom/train.txt','w') 
f2 = open('/content/darknet/custom/validation.txt','w') 
while True : 
  if i == 0 : 
    line = txt.readline() 
    if not line : 
      break 
    count +=1 
    if count < int(length/10)*2 : 
      f2.write(line) 
    else : 
      f.write(line) 
txt.close() 
f.close() 
f2.close()

첫 학습에 사용할 초기 모델로 yolov4.conv.137 을 가져옵니다.

In [ ]:
!gdown --id 1JKF-bdIklxOOVy-2Cr5qdvjgGpmGfcbp -O /content/darknet/yolov4.conv.137

Downloading...
From: https://drive.google.com/uc?id=1JKF-bdIklxOOVy-2Cr5qdvjgGpmGfcbp
To: /content/darknet/yolov4.conv.137
170MB [00:01, 85.3MB/s]


In [ ]:
%cd /content/darknet

/content/darknet


YOLOv4 학습을 시작합니다.

In [ ]:
!./darknet detector train custom/custom.data cfg/yolov4.cfg yolov4.conv.137 -map -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.544892), count: 2, class_loss = 0.731573, iou_loss = 1.107013, total_loss = 1.838586 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.493130), count: 12, class_loss = 3.468613, iou_loss = 1.354803, total_loss = 4.823415 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.485196), count: 11, class_loss = 4.087059, iou_loss = 0.213727, total_loss = 4.300785 
 total_bbox = 405787, rewritten_bbox = 0.773559 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 1, class_loss = 0.000251, iou_loss = 0.000000, total_loss = 0.000251 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.474780), count: 12, class_loss = 4.427674, iou_loss = 0.982150, total_loss = 5.409824 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Regio